<h3>First-Class Functions</h3>

Functions in Python are first-class objects. Programming language theorists define a “first-class object” as a program entity that can be:

* Created at runtime
* Assigned to a variable or element in a data structure
* Passed as an argument to a function
* Returned as the result of a function

Integers, strings, and dictionaries are other examples of first-class objects in Python. This chapter focuses on the implications of practical applications of treating functions as objects.

In [1]:
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n * factorial(n-1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'returns n!'

In [4]:
type(factorial)

function

In [5]:
# assign var to function, accessing it through a different name

fact = factorial
fact

<function __main__.factorial(n)>

In [6]:
fact(5)

120

In [7]:
map(factorial, range(11))

In [8]:
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

<h3>Higher Order Functions</h3>

A function that takes a function as argument or returns a function as the result is a <i>higher-order function</i>. One example is `map`. Another is the built in function `sorted`: an optional `key` arg lets you provide a function to be applied to each item for sorting.

In the functional programming paradigm, some of the best known higher-order functions are `map`, `filter`, `reduce`, and `apply`. `apply` isn't part of Python anymore, and `map, filter, reduce` have modern replacements (listcomp).

Note that `map` and `fitler` return generators -- a form of iterator -- so their direct substitute is now a generator expression. 

The `reduce` function was demoted from a built-in in Python 2 to the `functools` module in Python 3. Its most common use case was summation, which is better served by using the `sum` built-in function. 

In [9]:
# Any one-arg function can be used as the key 

fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [10]:
def reverse(word):
    return word[::-1]

reverse('testing')

'gnitset'

In [11]:
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

In [12]:
# Replacement for map and filter is listcomp

list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [13]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [14]:
list(map(factorial, filter(lambda n : n % 2, range(6))))

[1, 6, 120]

In [15]:
[factorial(n) for n in range(6) if n % 2] # don't need map, filter, or lambda

[1, 6, 120]

In [16]:
from functools import reduce
from operator import add

reduce(add, range(100))

4950

In [17]:
sum(range(100)) # way better way to do this

4950

In [18]:
# any and all are other reducing built-ins

bools = [True, False, True]
all(bools)

False

In [19]:
any(bools)

True

<h3>Anonymous Functions</h3>

The `lambda` keyword creates an anonymous function with a Python expression.

The body of a `lambda` function must be a pure expression -- that is, it cannot make assignments or use any other Python statements such as `while`, `try`, etc.

The best use of anonymous functions is in the context of an argument list. For example, the rhyme index below is rewritten with lambda, without using a reverse function.

Outside the limited context of arguments to higher-order functions, anonymous functions are rarely used in Python. The syntactic restrictions tend to make nontrivial lambdas either unreadable or unworkable.

**If a lamabda function is difficult to understand, re-write it as an actual function**

In [20]:
sorted(fruits, key=lambda word : word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

<h3>The Seven Flavors of Callable Objects</h3>

1. User-defined functions

Created with def statements or lambda expressions.

2. Built-in functions

A function implemented in C (for CPython), like len or time.strftime.

3. Built-in methods

Methods implemented in C, like dict.get.

4. Methods

Functions defined in the body of a class.

5. Classes

When invoked, a class runs its `__new__` method to create an instance, then `__init__` to initialize it, and finally the instance is returned to the caller. Because there is no new operator in Python, calling a class is like calling a function. (Usually calling a class creates an instance of the same class, but other behaviors are possible by overriding `__new__`.

6. Class instances

If a class defines a `__call__` method, then its instances may be invoked as functions.

7. Generator functions

Functions or methods that use the `yield` keyword. When called, generator functions return a generator object.

<h3>User-Defined Callable Types</h3>

Python functions are real objects, and arbitrary Python objects may also be made to behave like functions. Implementing a `__call__` instance method is all it takes.

Below we implement a `BingoCage` class. An instance is built from any iterable, and stores an internal list of items, in random order. Calling the instance pops an item.

In [21]:
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('Picked from empty BingoCage')
    
    def __call__(self): # shortcut to bingo.pick() becomes bingo()
        return self.pick()

In [22]:
bingo = BingoCage(range(3))
bingo.pick()

0

In [23]:
bingo()

1

In [24]:
callable(bingo)

True

A class implementing `__call__` is an easy way to create function-like objects that have some internal state that must be kept across invocations, like the remaining items in the `BingoCage`. An example is a decorator. Decorators must be functions, but it is sometimes convenient to be able to "remember" something between calls of the decorator (e.g., for memoization -- caching the results of expensive computations for later use).

A totally different approach to creating functions with internal state is to use closures. Closures, as well as decorators, are covered in ch. 7.

<h3>Runtime Function Introspection</h3>

Function objects have many attributes beyond `__doc__`. See what the `dir` function reveals about our `factorial` function.

Most of these attributes are common to Python objects in general. In this section, we cover those that are especially relevant to treating functions as objects, starting with `__dict__`.

Like the instances of a plain user-defined class, a function uses the `__dict__` attribute to store user attributes assigned to it. This is useful as a primitive form of annotation.

In [25]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [26]:
def upper_case_name(obj):
    return (f'{obj.first_name.upper(), obj.last_name.upper()}')
upper_case_name.short_description = 'Customer name'

In [27]:
class CustomerName:
    
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

upper_case_name(CustomerName('John', 'Smith'))

"('JOHN', 'SMITH')"

In [28]:
# Look at the difference between function and generic user-defined object

class C:
    pass
obj = C()

def func():
    pass

# These exist in function, but not in object
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

In [29]:
# This exists in object, but not function

sorted(set(dir(obj)) - set(dir(func)))

['__weakref__']

In [30]:
# Positional and Keyword Parameters

def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s=%s' % (attr, value)
                            for attr, value
                            in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s<%s>' %
                        (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [31]:
tag('br')

'<br />'

In [32]:
tag('p', 'hello')

'<p>hello<p>'

In [33]:
print(tag('p', 'hello', 'world'))

<p>hello<p>
<p>world<p>


In [34]:
tag('p', 'hello', id=33)

'<p id=33>hello<p>'

In [35]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class=sidebar>hello<p>
<p class=sidebar>world<p>


In [36]:
tag(content='testing', name='img')

'<img content=testing />'

In [37]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
          'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class=framed src=sunset.jpg title=Sunset Boulevard />'

In [38]:
# To specify keyword-only args, name them after the arg specified with *

def f(a, *, b): # with no default value, this is a required arg
    return a, b

f(1, b=2)

(1, 2)

In [39]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [40]:
clip.__defaults__ # shows defaults in reverse order

(80,)

In [41]:
clip.__code__

<code object clip at 0x7f96f002d030, file "<ipython-input-39-35ad83a19ae6>", line 1>

In [42]:
clip.__code__.co_varnames # first N are arg names, we know N from the cell below

('text', 'max_len', 'end', 'space_before', 'space_after')

In [43]:
clip.__code__.co_argcount # this tells us that there are 2 function args

2

In [44]:
# Better way to inspect the module

from inspect import signature

sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [45]:
str(sig)

'(text, max_len=80)'

In [46]:
# Much more clear than inspecting with the __magic__ methods

for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [47]:
import inspect

sig = inspect.signature(tag) # create inspect.BoundArguments object 
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag) # pass the dictionary of args to .bind()
bound_args # inspect.BoundArgument object

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [48]:
for name, value in bound_args.arguments.items(): # iterate over OrderedDict to display names and args
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [49]:
del my_tag['name'] # remove required arg from my_tag
bound_args = sig.bind(**my_tag) # TypeError due to missing name parameter

TypeError: missing a required argument: 'name'

In [50]:
# Function Annotations allow you to attach metadata to parameters of a function declaration
# and its return values. These are stored in the __annotations__ attribute of a function

def clip(text:str, max_len:'int > 0'=80) -> str:
    """Return text clipped at the last space before or after max_len """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before 
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [51]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

As of now, there are no checks, enforcement, validation, or any other action being performed with the annotations. They have no meaning to the Python interpreter. They are just metadata that may be used by tools, such as IDEs, frameworks, and decorators.

In the future, frameworks such as Bobo could support annotations to further automate request processing. For example, an arg annotated as `price:float` may be automatically converted from a query string to `float`. The biggest impact would probably be for static type checking in tools like IDEs and linters, not dynamic settings like Bobo.

In [52]:
# There is no equivalent of the sum() function for multiplication, so we use reduce with lambda

from functools import reduce

def fact(n):
    return reduce(lambda a, b : a * b, range(1, n + 1))

In [53]:
# Same thing as above, without the need for anonymous lambda function

from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n + 1))

In [54]:
metro_data = [
('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [55]:
from operator import itemgetter

# itemgetter does the same as lambda fields : fields[1]
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [56]:
# itemgetter will return tuples with extracted values when passed multiple index args

cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


In [57]:
# attrgetter extracts object attributes by name

from collections import namedtuple

LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', ' name cc pop coord')

metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
              for name, cc, pop, (lat, long) in metro_data]
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [58]:
metro_areas[0].coord.lat

35.689722

In [59]:
from operator import attrgetter

name_lat = attrgetter('name', 'coord.lat')

for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [60]:
# List of functions defined in operator

import operator

', '.join([name for name in dir(operator) if not name.startswith('_')])

'abs, add, and_, attrgetter, concat, contains, countOf, delitem, eq, floordiv, ge, getitem, gt, iadd, iand, iconcat, ifloordiv, ilshift, imatmul, imod, imul, index, indexOf, inv, invert, ior, ipow, irshift, is_, is_not, isub, itemgetter, itruediv, ixor, le, length_hint, lshift, lt, matmul, methodcaller, mod, mul, ne, neg, not_, or_, pos, pow, rshift, setitem, sub, truediv, truth, xor'

In [61]:
# methodcaller is similar to attrgetter and itemgetter in that it creates a function on the fly.
# The function it creates calls a method by name on the object given as argument.

from operator import methodcaller

s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [62]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

In [63]:
str.upper(s)

'THE TIME HAS COME'

<h3>Freezing Arguments with functools.partial</h3>

The `functools` module brings together a handful of higher-order functions. `reduce` is probably the best known. `functools.partial` is a higher-order function that allows partial application of a function. Given a function, a partial application produces a new callable with some of the arguments of the original function fixed. This is useful to adapt a function that takes one or more arguments to an API that requires a callback with fewer arguments.

In [64]:
from operator import mul
from functools import partial

triple = partial(mul, 3)
triple(7)

21

In [65]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [66]:
# A more useful example involves the unicode.normalize function

import unicodedata, functools

nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [67]:
s1 == s2

False

In [68]:
nfc(s1) == nfc(s2)

True

An impressive `functools` function is `lru_cache`, which does memoization -- a form of automatic optimization that works by storing the results of function calls to avoid expensive recalculations. We'll use this in the decorators chapter, along with other higher-order functions.